In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%qtconsole

In [36]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

batch_size = 1000
data, labels = [],[]
for i in range(int(x_train.shape[0]/batch_size)):
    data.append(x_train[i*batch_size:batch_size*(i+1),:,:])
    labels.append(y_train[i*batch_size:batch_size*(i+1)])
train_ds = [[data[i],labels[i]] for i in range(len(data))]

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(120, activation='sigmoid'),
    tf.keras.layers.Dense(80, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
model.fit(x_train, y_train, epochs=2)

Train on 60000 samples
Epoch 1/2
60000/60000 [==============================] - 8s 126us/sample - loss: 1.0586 - accuracy: 0.6942
Epoch 2/2
60000/60000 [==============================] - 8s 127us/sample - loss: 0.3486 - accuracy: 0.9007


In [6]:
model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 1s - loss: 0.2206 - accuracy: 0.9155


[0.2952850646972656, 0.9155]

In [7]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model

In [67]:
class MnistClassifier(Model):
    def __init__(self):
        super(MnistClassifier,self).__init__()
        self.layer0 = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.layer1 = Dense(120, activation='sigmoid')
        self.layer2 = Dense(80, activation='sigmoid')
        self.layer3 = Dense(10, activation='sigmoid')
        
    def call(self,x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [68]:
classifier = MnistClassifier()
loss_object = tf.keras.losses.MeanSquaredError()
sgd = tf.keras.optimizers.SGD(lr=0.05,momentum=0.9)

In [69]:
@tf.function
def train_step(data,labels):
    with tf.GradientTape() as tape:
        predictions = classifier(data)
        labels = tf.one_hot(labels,10)
        loss = loss_object(labels,predictions)
    gradients = tape.gradient(loss, classifier.trainable_variables)
    sgd.apply_gradients(zip(gradients, classifier.trainable_variables))

In [70]:
EPOCHS = 1000

for epoch in range(EPOCHS):
    for data,labels in train_ds:
        train_step(data,labels)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [71]:
def mlpclass(y):
    #Retourne les classes de la sortie y 
    if len(y.shape)==1:
        res = np.argmax(y)
    else: 
        res = []
        for i in range(y.shape[1]):
            res.append(np.argmax(y[:,i]))
    return res

In [72]:
predictions = classifier(x_train)
predictions = predictions.numpy()
n,m = predictions.shape
for i in range(n):
    for j in range(m):
        predictions[i,j] = round(predictions[i,j])
predictions = mlpclass(predictions.T)

In [73]:
def score(label, pred):
    #Retourne le score de la prédiction
    verification = label == pred
    res = [e for e in verification if e ==True]
    nb = len(res)
    taux = round(len(res)/len(verification),2)
    return {'nb': nb,'taux': taux }

In [74]:
score(y_train,predictions)

{'nb': 55726, 'taux': 0.93}